# Proteasome proteins' expression in tumor vs. normal samples

This notebook uses permutation testing to investigate the abundance levels of proteins making up the proteasome in tumor vs. normal samples.

Import needed packages.

In [1]:
import cptac
import pcanalyzer as pc
import seaborn as sns
import matplotlib.pyplot as plt

Load the datasets.

In [ ]:
datasets = []

#datasets.append(cptac.Brca()) # Brca has no normal samples
datasets.append(cptac.Ccrcc())
#datasets.append(cptac.Colon())
#datasets.append(cptac.Endometrial())

#datasets.append(cptac.Gbm())
#datasets.append(cptac.Hnscc())

#datasets.append(cptac.Lscc())
#datasets.append(cptac.Luad())

#datasets.append(cptac.Ovarian())

Get a list of proteins within the proteasome.

In [66]:
protein_lists = pc.get_member_proteins()
proteasome_proteins = protein_lists['PA700 complex'] + protein_lists['26S proteasome'] + protein_lists['20S proteasome'] + protein_lists['PA28-20S proteasome'] + protein_lists['PA700-20S-PA28 complex'] + protein_lists['PA28gamma-20S proteasome']
proteasome_proteins = sorted(set(proteasome_proteins))
proteasome_proteins

['PSMA1',
 'PSMA2',
 'PSMA3',
 'PSMA4',
 'PSMA5',
 'PSMA6',
 'PSMA7',
 'PSMB1',
 'PSMB2',
 'PSMB3',
 'PSMB4',
 'PSMB5',
 'PSMB6',
 'PSMB7',
 'PSMC1',
 'PSMC2',
 'PSMC3',
 'PSMC4',
 'PSMC5',
 'PSMC6',
 'PSMD1',
 'PSMD10',
 'PSMD11',
 'PSMD12',
 'PSMD13',
 'PSMD14',
 'PSMD2',
 'PSMD3',
 'PSMD4',
 'PSMD5',
 'PSMD6',
 'PSMD7',
 'PSMD8',
 'PSMD9',
 'PSME1',
 'PSME2',
 'PSME3']